In [13]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col

# 1️⃣ Διαβάζουμε Silver
silver_df = spark.read.table("silver_sales")

# 2️⃣ Προετοιμάζουμε Gold DataFrame
gold_df = silver_df.select(
    col("SaleID"),
    col("CustomerID"),
    col("Product"),
    col("Quantity"),
    col("UnitPrice"),
    col("SaleDate"),
    col("UpdatedDate")
)

# 3️⃣ Όνομα Gold table
gold_table_name = "gold_sales"

# 4️⃣ Έλεγχος existence Gold
if spark.catalog.tableExists(gold_table_name):
    # Δημιουργούμε DeltaTable object
    gold_table = DeltaTable.forName(spark, gold_table_name)
    
    # MERGE
    gold_table.alias("target") \
        .merge(
            gold_df.alias("source"),
            "target.SaleID = source.SaleID"
        ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    # Δεν υπάρχει → δημιουργία Gold
    gold_df.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable(gold_table_name)


StatementMeta(, 3b57584e-3b25-4a9c-a51b-b084eb6b41c9, 15, Finished, Available, Finished)